In [125]:
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras.utils.np_utils import to_categorical
import tensorflow as tf

# Using box score, also games won this season and last season by team 


# DATA PREPROCESSING
x = pd.read_csv('BBRefMVPData.csv')
y = ['Allen Iverson', 'Tim Duncan', 'Tim Duncan', 'Kevin Garnett', 'Steve Nash', 'Steve Nash', \
    'Dirk Nowitzki', 'Kobe Bryant', 'LeBron James', 'LeBron James', 'Derrick Rose', 'LeBron James', \
    'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', \
    'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokić']

train_amount = int(0.6 * len(y))

y_train = []
y_test = []
y_pred = []
sample_weights = [] # Penalty matrix to address class imbalance

x = x.assign(Year=pd.Series(np.random.randn(len(x))).values)
x.columns = ['Number', 'Name', 'Games', 'FG%', '3P%', 'eFG%', 'FT%', 'Rebounds', 'Assists', 'Steals', 'Blocks', \
                   'Turnovers', 'Points', 'Seed', 'Impact', 'Year']
x.fillna(0, inplace=True)

indices = x.loc[:, 'Number']
targetLen = 525
featureLen = len(x.columns) - 2
regularGameAmount = 82
firstIndex = 0
secondIndex = 0
splitIndex = 0
predIndex = 0
i = 0
start_year = 2001
pred_year = 2021
year = start_year
shortenedGames = {2012: 66, 2020: 72.5, 2021: 72}

while i < len(x):
    if x.loc[i, 'Number'] == 0 and i != 0:
        year += 1
        # Splitting data into its sections
        if year - start_year == train_amount:
            splitIndex = i
        if year == pred_year:
            predIndex = i
    # Making sure games played isn't skewed by lockout-shortened or COVID-impacted seasons
    if year == 2012 or year == 2020 or year == 2021:
        x.loc[i, 'Games'] = int(round((x.loc[i, 'Games'] / shortenedGames[year]) * regularGameAmount))
    # Assembling target vectors with 1s for MVPs
    y_val = 1 if (x.loc[i, 'Name'] == y[year-start_year]) else 0
    if year - start_year < train_amount:
        y_train.append(y_val)
    elif year == pred_year:
        y_pred.append(y_val)
    else:
        y_test.append(y_val)
    # Putting together weights for penalty matrix only for players who meet the Games Played threshold
    if splitIndex == 0 and x.loc[i, 'Games'] >= 0.70 * regularGameAmount:
        weight = 1 if y_val == 0 else 10
        sample_weights.append(weight)
    # Adding a year column
    x.loc[i, 'Year'] = year
    i += 1

# Splitting data
x_train = x.iloc[0:splitIndex, 2:len(x.columns)]
x_train_info = x.iloc[0:splitIndex, 0:2] # Contains 'Number' and 'Name' columns
x_test = x.iloc[splitIndex:predIndex, 2:len(x.columns)]
x_test_info = x.iloc[splitIndex:predIndex, 0:2]
x_pred = x.iloc[predIndex:, 2:len(x.columns)]
x_pred_info = x.iloc[predIndex:, 0:2]

# Ensuring only those who played enough games are considered 
def adjustedForGames(df, target):
    df['Target'] = target
    df.drop(df[(df['Games'] < 0.70 * regularGameAmount)].index, inplace=True)
    newTarget = df['Target']
    df.drop('Target', 1, inplace=True)
    return newTarget

y_train = adjustedForGames(x_train, y_train)
y_test = adjustedForGames(x_test, y_test)
y_pred = adjustedForGames(x_pred, y_pred)

2733


In [126]:
# Setting up training and test data
# while i < len(indices):
#     if i == len(indices) - 1:
#         x_test.append(x[firstIndex:].iloc[:, 2:len(x.columns)])
#         x_test_info.append(x[firstIndex:].iloc[:, 0:2])
#     elif indices[i] == 0 and i != 0:
#         secondIndex = i
#         if len(x_train) == train_amount:
#             x_test.append(x[firstIndex:secondIndex].iloc[:, 2:len(x.columns)])
#             x_test_info.append(x[firstIndex:].iloc[:, 0:2])
#         else:
#             x_train.append(x[firstIndex:secondIndex].iloc[:, 2:len(x.columns)])
#             x_train_info.append(x[firstIndex:].iloc[:, 0:2])
#         firstIndex = secondIndex
#     i += 1

# Setting up target vectors with a 1 at the index equal to the 'Number' column value of the MVP
# year = 0
# while year < len(x_train) + len(x_test):
#     if year < train_amount:
#         players = x_train_info[year]
#         index = players[players['Name'] == y.loc[year, 0]].index.values
#         number = players.loc[index[0], 'Number']
#         y_train.append(np.concatenate((np.zeros((targetLen, 1)), np.ones((targetLen, 1))), axis=1))
#         y_train[-1][number][0] = 1
#         y_train[-1][number][1] = 0
#     else:
#         players = x_test_info[year-train_amount]
#         index = players[players['Name'] == y.loc[year, 0]].index.values
#         number = players.loc[index[0], 'Number']
#         y_test.append(np.zeros((targetLen, 1)))
#         y_test[-1][number] = 1
#     year += 1  

# year = 0
# playerNumsTrain = []
# playerNumsTest = []
# while year < len(x_train) + len(x_test):
#     if year < train_amount:
#         players = x_train_info[year]
#         index = players[players['Name'] == y.loc[year, 0]].index.values
#         number = players.loc[index[0], 'Number']
#         playerNumsTrain.append(number)
#     else:
#         players = x_test_info[year-train_amount]
#         index = players[players['Name'] == y.loc[year, 0]].index.values
#         number = players.loc[index[0], 'Number']
#         playerNumsTest.append(number)
#     year += 1

# y_train = to_categorical(playerNumsTrain, num_classes=targetLen)
# y_test = to_categorical(playerNumsTest, num_classes=targetLen)

# Adding dummy rows with a random low-value player to ensure uniform length of inputs w/o influencing the data
# dummy = x_train[0].loc[440, :]
# for yearly in x_train:
#     i = len(yearly)
#     while len(yearly) < targetLen:
#         yearly.loc[i, :] = dummy
#         i += 1

# for yearly in x_test:
#     i = len(yearly)
#     while len(yearly) < targetLen:
#         yearly.loc[i, :] = dummy
#         i += 1

In [134]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# RF MODEL
# model = RandomForestClassifier(max_depth=3, min_samples_leaf=6)
# cv = RepeatedStratifiedKFold()
# scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=1, error_score='raise')
# model.fit(x_train, y_train, sample_weight=sample_weights)
# prediction = model.predict(x_pred)

print(len(x_train))
print(len(y_train))

# XGBoost Model 
model = XGBClassifier(n_estimators=5, max_depth=3, learning_rate=1, max_delta_step=0.5, subsample=0.8)
cv = RepeatedStratifiedKFold()
scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=1, error_score='raise')
model.fit(x_train, y_train, sample_weight=sample_weights)
prediction = model.predict(x_pred)

i = 0
mvps = []
while i < len(prediction):
    if prediction[i] == 1:
        print(i)
        mvps.append(x_pred_info.iloc[i, 1])
    i += 1

print(prediction)
print(mvps)

# DL MODEL
# model = models.Sequential()
# model.add(layers.LSTM(targetLen))
# model.add(layers.Dense(2, activation='sigmoid'))
# model.build((targetLen, featureLen))
# model.summary()

# x_train_values = []
# for entry in x_train:
#     x_train_values.append(entry.to_numpy())
    
# print(y_train)

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
# model.fit(x_train, y_train, epochs=10, batch_size=20)




2733
2733
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the

/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of 

[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of 

[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of 

[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:28:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
